In [ ]:
from __future__ import absolute_import
from __future__ import division

import os
import numpy
import itertools
from sklearn.utils import shuffle

import pandas as pd

# Execute rpy2_setup.sh to install dependencies
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri

import tensorflow as tf

pandas2ri.activate()
numpy2ri.activate()

# Load RData as Pandas DataFrame

In [ ]:
fault_free_training_rf = robjects.r['load']("../data/TEP_FaultFree_Training.RData")
fault_free_testing_rf = robjects.r['load']("../data/TEP_FaultFree_Testing.RData")
fault_free_training_df = shuffle(pandas2ri.ri2py_dataframe(robjects.globalenv["fault_free_training"]))
fault_free_testing_df = shuffle(pandas2ri.ri2py_dataframe(robjects.globalenv["fault_free_testing"]))

# Cleanup RData variables
del fault_free_training_rf
del fault_free_testing_rf

In [ ]:
faulty_training_rf = robjects.r['load']("../data/TEP_Faulty_Training.RData")
faulty_testing_rf = robjects.r['load']("../data/TEP_Faulty_Testing.RData")
faulty_training_df = shuffle(pandas2ri.ri2py_dataframe(robjects.globalenv["faulty_training"]))
faulty_testing_df = shuffle(pandas2ri.ri2py_dataframe(robjects.globalenv["faulty_testing"]))

# Cleanup RData variables  
del faulty_training_rf
del faulty_testing_rf

---------------------------------------------------------------------------------------------------------

# Get Data Stream Iterator

In [ ]:
def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=data_set.drop(['faultNumber', 'simulationRun', 'sample'], axis=1),
      y = data_set['faultNumber'],
      num_epochs=num_epochs,
      shuffle=shuffle)

In [ ]:
def get_feature_cols(data_set):
    return [tf.feature_column.numeric_column(k) 
            for k in data_set.drop(['faultNumber', 'simulationRun', 'sample'], axis=1).keys()]

# Simple Neural Network

In [ ]:
# train_df = shuffle(pd.concat([fault_free_training_df, faulty_training_df]))
# test_df = shuffle(pd.concat([fault_free_testing_df, faulty_testing_df]))
train_df = faulty_training_df.copy(deep=False)
test_df = faulty_testing_df.copy(deep=False)

# Cleanup DataFrames
# del fault_free_testing_df
# del fault_free_training_df
# del faulty_testing_df
# del faulty_training_df

In [ ]:
regressor = tf.estimator.DNNRegressor(feature_columns=get_feature_cols(train_df), hidden_units=[10, 10])

In [ ]:
regressor.train(get_input_fn(train_df, 10), steps=500)

In [ ]:
y = regressor.predict(input_fn=get_input_fn(test_df[:10], num_epochs=1, shuffle=False))
predictions = list(p["predictions"] for p in itertools.islice(y, 6))

In [ ]:
print "Prediction: " + str([int(i[0]) for i in predictions])
print "Actual: " + str(test_df['faultNumber'][:10].values)